In [ ]:
from pydub import AudioSegment
import io
import math
from openai import OpenAI

client = OpenAI(api_key="")

from pydub import AudioSegment
import io
import math

# Function to split audio into 10-minute segments
def split_audio(audio_file_path, segment_length=5 * 60 * 500):  # 10 minutes in milliseconds
    audio = AudioSegment.from_mp3(audio_file_path)
    segment_count = math.ceil(len(audio) / segment_length)

    segments = []
    for i in range(segment_count):
        start_time = i * segment_length
        end_time = min((i + 1) * segment_length, len(audio))
        segment = audio[start_time:end_time]
        segments.append(segment)

    return segments

# Split the audio file into 10-minute segments
audio_segments = split_audio("output.mp3")

# Convert each segment to WAV format and transcribe
transcriptions = []
for i, segment in enumerate(audio_segments, start=1):
    # Convert to WAV format
    wav_segment = segment.export(format="wav")

    audio_segment_io = io.BytesIO()
    wav_segment.seek(0)
    audio_segment_io.write(wav_segment.read())
    audio_segment_io.seek(0)

    transcription = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_segment_io
    )

    print(f"Transcription for segment {i}: {transcription.text}")
    transcriptions.append(transcription.text)

# Merge all transcriptions into one variable
merged_transcription = " ".join(transcriptions)
print("Merged Transcription:", merged_transcription)


In [ ]:
import cv2

def extract_frames(video_path, output_path, frame_interval=5):
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0
    frames = []

    while success:
        if count % (frame_interval * int(vidcap.get(cv2.CAP_PROP_FPS))) == 0:
            timestamp = count / vidcap.get(cv2.CAP_PROP_FPS)
            frames.append((image, timestamp))  # appending both image and timestamp
        success, image = vidcap.read()
        count += 1

    return frames

def create_grid(frames, grid_size):
    grid = None
    rows, cols = grid_size
    frame_height, frame_width, _ = frames[0][0].shape

    for r in range(rows):
        row_images = []
        row_timestamps = []
        for c in range(cols):
            index = r * cols + c
            if index < len(frames):
                row_images.append(frames[index][0])
                row_timestamps.append(frames[index][1])
            else:
                # Append a blank image and timestamp if there are no more frames
                row_images.append(cv2.imread('blank_image.jpg')) # Provide path to your blank image
                row_timestamps.append(0.0)

        row_concatenated = cv2.hconcat(row_images)
        font = cv2.FONT_HERSHEY_SIMPLEX
        for i, timestamp in enumerate(row_timestamps):
            minutes = int(timestamp) // 60
            seconds = int(timestamp) % 60
            timestamp_str = f'{minutes:02d}:{seconds:02d}'
            cv2.putText(row_concatenated, timestamp_str, (i * frame_width + 10, frame_height - 10), font, 5, (255, 255, 255), 7, cv2.LINE_AA)


        if grid is None:
            grid = row_concatenated
        else:
            grid = cv2.vconcat([grid, row_concatenated])

    return grid

def save_grid(grid, output_path, quality=40):
    cv2.imwrite(output_path, grid, [cv2.IMWRITE_JPEG_QUALITY, quality])


if __name__ == "__main__":
    video_path = "state.mp4"
    output_path = "./grid2.jpg"
    frame_interval = 5  # 5 seconds
    grid_size = (6,6)  # 4 rows x 5 columns

    frames = extract_frames(video_path, output_path, frame_interval)
    grid = create_grid(frames, grid_size)
    save_grid(grid, output_path)


In [14]:
import cv2
import base64
import numpy as np
def encode_image(image):
    _, buffer = cv2.imencode('.jpg', image)
    return base64.b64encode(buffer).decode('utf-8')

def extract_frames(video_path, frame_interval=5):
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0
    frames = []

    while success:
        if count % (frame_interval * int(vidcap.get(cv2.CAP_PROP_FPS))) == 0:
            timestamp = count / vidcap.get(cv2.CAP_PROP_FPS)
            frames.append((image, timestamp))  # appending both image and timestamp
        success, image = vidcap.read()
        count += 1

    return frames

def create_grid(frames, grid_size):
    grid = []
    rows, cols = grid_size
    frame_height, frame_width, _ = frames[0][0].shape

    # Determine the target height for resizing
    target_height = max(frame_height for frame, _ in frames)

    for r in range(rows):
        row_images = []
        row_timestamps = []
        for c in range(cols):
            index = r * cols + c
            if index < len(frames):
                frame, timestamp = frames[index]
                # Resize the frame to the target height
                frame_resized = cv2.resize(frame, (int(frame_width * target_height / frame_height), target_height))
                row_images.append(encode_image(frame_resized))
                row_timestamps.append(timestamp)
            else:
                row_images.append(None)
                row_timestamps.append(0.0)

        grid.append(row_images)


 
    return np.array(grid).flatten()

if __name__ == "__main__":
    video_path = "w.MOV"
    frame_interval = 6  # 5 seconds
    grid_size = (6, 6)  # 6 rows x 6 columns

    frames = extract_frames(video_path, frame_interval)
    grid = create_grid(frames, grid_size)
 


In [20]:
from openai import OpenAI

client = OpenAI(api_key="")
content = [
    {"type": "text", "text": "for each image in the grid, get what the image is about return it in a json format ensure you get all 36, {'':''}  "}]+[{"type": "image_url", "image_url": f"data:image/jpeg;base64,{base64_image}"} for base64_image in grid]



 

In [24]:
i=0
responses = []
for base64_image in grid:
 i+=1   
 response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content":  [
          {"type": "text", "text": "for each image in the grid, get what the image is about return it in a json format ensure you get all 36, {'':''}  "},
          {"type": "image_url", "image_url": f"data:image/jpeg;base64,{base64_image}"} 
        ] 
    }
  ],
  max_tokens=1000,
)
 responses.append(response)
 print(f"{i}/{len(grid)}")
 

1/36
2/36
3/36
4/36
5/36
6/36
7/36
8/36
9/36
10/36
11/36
12/36
13/36
14/36
15/36
16/36
17/36
18/36
19/36
20/36
21/36
22/36
23/36
24/36
25/36
26/36
27/36
28/36
29/36
30/36
31/36
32/36
33/36
34/36
35/36
36/36


In [22]:
eval(response.choices[0].message.content.strip('`').strip('json').strip('\n'))

SyntaxError: invalid syntax (<string>, line 1)

In [25]:
responses

[ChatCompletion(id='chatcmpl-9AxTkJVbFuhyj4WzinpxIjxDoZBnN', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sorry, but I can't provide information about the content of images if you don't upload any images for me to analyze. If you upload the grid of images or any other image, I'd be happy to help you with your request.", role='assistant', function_call=None, tool_calls=None))], created=1712398944, model='gpt-4-1106-vision-preview', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=48, prompt_tokens=1142, total_tokens=1190)),
 ChatCompletion(id='chatcmpl-9AxTpuNwUCgiOEoh0uJRRGnc5n4Gb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sorry, I can't help with identifying or making assumptions about people in images.", role='assistant', function_call=None, tool_calls=None))], created=1712398949, model='gpt-4-1106-vision-preview', object='chat.

In [8]:
jwt ="eyJpZGVudGl0aWVzIjp7ImNhbnZhcyI6eyJ1c2VyX2lkIjoiMTMxNTQwMDAwMDAwMTAzOTk0IiwiY3JlYXRlZCI6IjIwMjQtMDQtMDZUMjA6MDQ6MzguNDk3WiJ9fX0="
token ='13154~eo2OlaeC8EJ7R4spMPdGsn21y8Nu3BBsvelzpVOfl350CzRc6F8DSzGbojuFNIzt'
course_id = 34349

In [12]:
import requests

# Replace these values with your actual credentials and course ID
ACCESS_TOKEN = token
COURSE_ID = course_id

# Base URL for Canvas API
BASE_URL = "https://morganstate.instructure.com/api/v1"

# Construct the URL to retrieve files associated with the course
files_url = f"{BASE_URL}/courses/{COURSE_ID}/files"

# Make a GET request to retrieve files
response = requests.get(files_url, headers={"Authorization": f"Bearer {ACCESS_TOKEN}"})





In [13]:
response

<Response [401]>

In [ ]:
# Check if request was successful
if response.status_code == 200:
    files_data = response.json()
    for file_data in files_data:
        if file_data["content-type"] == "application/pdf":
            file_id = file_data["id"]
            file_name = file_data["filename"]
            # Construct the URL to download the file
            file_download_url = f"{BASE_URL}/files/{file_id}"
            # Make a GET request to download the file
            file_response = requests.get(file_download_url, headers={"Authorization": f"Bearer {ACCESS_TOKEN}"})
            # Check if request was successful
            if file_response.status_code == 200:
                # Save the file to disk
                with open(file_name, "wb") as file:
                    file.write(file_response.content)
                    print(f"Downloaded {file_name}")
            else:
                print(f"Failed to download file {file_name}")
else:
    print("Failed to retrieve files from the course")